In [1]:
from PyQt5 import QtGui, QtCore, QtWidgets
import numpy as np
# from HelperFunctions import *

class KeyPressEater(QtCore.QObject):
    '''
    This class can be used to intercept all key presses happening in the application
    and then choose to handle it or relay it to the application.
    All Events will be passed to the given handleEventFunction.
    It should return True if the event was handled otherwise False
    '''
    def __init__(self, handleEventFunction, *args, **kwargs):
        super().__init__(*args, **kwargs)
        app = QtWidgets.QApplication.instance()  # get current application
        app.installEventFilter(self)
        self.handleEventFunction = handleEventFunction

    def eventFilter(self, obj: QtCore.QObject, event: QtCore.QEvent) -> bool:
        return self.handleEventFunction(event)
        


class ArenaGraphicsPathItem(QtWidgets.QGraphicsPathItem):
    '''
    A QGraphicsPathItem that is connected to a QGraphicsScene and two QDoubleSpinBoxes
    that hold the position of this item.
    '''

    def __init__(self, parentWidget, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.parentWidget = parentWidget
        self.setFlags(
            QtWidgets.QGraphicsItem.GraphicsItemFlag.ItemIsSelectable |
            QtWidgets.QGraphicsItem.GraphicsItemFlag.ItemSendsGeometryChanges
            )

        self.isDragged = False

        # create brush
        brush = QtGui.QBrush(QtGui.QColor("red"), QtCore.Qt.BrushStyle.SolidPattern)
        self.setBrush(brush)
        # create pen
        pen = QtGui.QPen()
        pen.setWidthF(0.01)
        pen.setStyle(QtCore.Qt.PenStyle.SolidLine)
        pen.setCapStyle(QtCore.Qt.PenCapStyle.RoundCap)
        pen.setJoinStyle(QtCore.Qt.PenJoinStyle.RoundJoin)
        self.setPen(pen)

        # add text item for displaying the name next to the item
        self.textItem = QtWidgets.QGraphicsTextItem("")
        self.textItem.setZValue(5)  # place text item above everything else
        self.textItem.setScale(0.035)
        parentWidget.graphicsScene.addItem(self.textItem)
        self.updateTextItemPos()

        # catch key presses for interacting with the item in the scene
        self.keyPressEater = KeyPressEater(self.handleEvent)
        
        self.oldItemPos = self.scenePos()

    # Alternative version of updateTextItemPos().
    # Calculate the bounding circle of the item and
    # place the text at an angle of 1/2 * PI.
    # def updateTextItemPos(self):
    #     # get radius of bounding circle
    #     rect = self.path().boundingRect()
    #     center = rect.center()
    #     point = rect.topLeft()
    #     diff = point - center
    #     radius = np.sqrt(diff.x() ** 2 + diff.y() ** 2)
    #     # get cartesian coordinates from polar coordinates
    #     x = np.cos((1/4) * 2 * np.pi) * radius
    #     y = np.sin((1/4) * 2 * np.pi) * radius
    #     # set text item pos
    #     pos = self.mapToScene(self.transformOriginPoint())
    #     pos += QtCore.QPointF(x, y)
    #     self.textItem.setPos(pos)

    def updateTextItemPos(self):
        pos = self.mapToScene(self.transformOriginPoint())
        # move a bit because the anchor position of text is top left
        pos += QtCore.QPointF(-0.3, -0.5)
        self.textItem.setPos(pos)

    def setPosNoEvent(self, x, y):
        self.setFlag(QtWidgets.QGraphicsItem.GraphicsItemFlag.ItemSendsGeometryChanges, False)
        super().setPos(x, y)
        self.setFlag(QtWidgets.QGraphicsItem.GraphicsItemFlag.ItemSendsGeometryChanges, True)

    def itemChange(self, change, value):
        if change == QtWidgets.QGraphicsItem.GraphicsItemChange.ItemPositionChange:
            self.updateTextItemPos()
            self.parentWidget.handleItemChange()

        return super().itemChange(change, value)

    def mousePressEvent(self, mouse_event):
        for item in self.scene().selectedItems():
            item.isDragged = True
        self.isDragged = True

        self.oldMousePos = mouse_event.scenePos()
        self.oldItemPos = self.scenePos()
        # save positions for all items in scene
        for item in self.scene().items():
            if hasattr(item, "oldItemPos"):
                item.oldItemPos = item.scenePos()
        self.oldItemRotation = self.rotation()
        modifier = QtWidgets.QApplication.keyboardModifiers()
        if modifier == QtCore.Qt.KeyboardModifier.ControlModifier:
            self.ctrlPressed = True
        else:
            self.ctrlPressed = False

        return super().mousePressEvent(mouse_event)

    def mouseMoveEvent(self, mouse_event):
        # rotate
        if self.isDragged and self.ctrlPressed:
            diff = mouse_event.scenePos() - self.oldMousePos
            angle = np.arctan2(diff.y(), diff.x())
            angle = 360 * angle / (2 * np.pi)  # convert radians to degrees
            self.setRotation(angle)
            self.updateTextItemPos()
        # translate
        elif self.isDragged:
            diff = mouse_event.scenePos() - self.oldMousePos
            for item in self.scene().selectedItems():
                if hasattr(item, "oldItemPos"):
                    item.setPos(item.oldItemPos + diff)

    def mouseReleaseEvent(self, mouse_event):
        for item in self.scene().selectedItems():
            if hasattr(item, "isDragged"):
                item.isDragged = False
        return super().mouseReleaseEvent(mouse_event)

    def mouseDoubleClickEvent(self, mouse_event):
        self.parentWidget.handleMouseDoubleClick()

    def handleEvent(self, event):
        # delete item when selected and DELETE key pressed
        if (event.type() == QtCore.QEvent.Type.KeyRelease
            and event.key() == QtCore.Qt.Key.Key_Delete
            and self.isSelected()):
            self.remove()

        # return false so event is not consumed and can be handled by other items aswell
        return False

    def remove(self):
        self.parentWidget.remove()



class ArenaGraphicsEllipseItem(QtWidgets.QGraphicsEllipseItem):
    '''
    A QGraphicsEllipseItem that is connected to two QDoubleSpinBoxes
    that hold the position of this item.
    '''

    def __init__(self, xSpinBox: QtWidgets.QDoubleSpinBox = None, ySpinBox: QtWidgets.QDoubleSpinBox = None, *args, handlePositionChangeMethod = None, **kwargs):
        """
        args:
            - xSpinBox: a spin box for the X-coordinate that shall be connected to this item
            - ySpinBox: a spin box for the Y-coordinate that shall be connected to this item
            - handlePositionChangeMethod: A method of the parent widget that should be called when this items position changes.
                It should take a QPointF as argument.
        """
        super().__init__(*args, **kwargs)
        self.setFlags(
            QtWidgets.QGraphicsItem.GraphicsItemFlag.ItemIsSelectable |
            QtWidgets.QGraphicsItem.GraphicsItemFlag.ItemSendsGeometryChanges
            )
        self.isDragged = False

        # set brush
        brush = QtGui.QBrush(QtGui.QColor("green"), QtCore.Qt.BrushStyle.SolidPattern)
        self.setBrush(brush)
        # set pen
        pen = QtGui.QPen()
        pen.setWidthF(0.01)
        pen.setStyle(QtCore.Qt.PenStyle.SolidLine)
        pen.setCapStyle(QtCore.Qt.PenCapStyle.RoundCap)
        pen.setJoinStyle(QtCore.Qt.PenJoinStyle.RoundJoin)
        self.setPen(pen)

        # text item for displaying the name next to the item, needs to be enabled by calling self.enableTextItem by parent
        self.textItem = QtWidgets.QGraphicsTextItem("")
        self.textItem.setScale(0.05)
        self.textItemEnabled = False

        self.xSpinBox = xSpinBox
        self.ySpinBox = ySpinBox

        self.handlePositionChangeMethod = handlePositionChangeMethod

        self.oldItemPos = self.scenePos()
        self.ctrlPressed = False

    def enableTextItem(self, scene: QtWidgets.QGraphicsScene, text: str):
        '''
        Add QGraphicsTextItem to scene and set its text.
        '''
        if not self.textItemEnabled:
            scene.addItem(self.textItem)
            self.textItem.setPlainText(text)
            self.textItemEnabled = True

    def updateTextItemPos(self):
        if self.textItemEnabled:
            self.textItem.setPos(self.scenePos())

    def setPosNoEvent(self, x, y):
        self.setFlag(QtWidgets.QGraphicsItem.GraphicsItemFlag.ItemSendsGeometryChanges, False)
        super().setPos(x, y)
        self.updateTextItemPos()
        self.setFlag(QtWidgets.QGraphicsItem.GraphicsItemFlag.ItemSendsGeometryChanges, True)

    def itemChange(self, change, value):
        if change == QtWidgets.QGraphicsItem.GraphicsItemChange.ItemPositionChange:
            if self.handlePositionChangeMethod != None:
                self.handlePositionChangeMethod(self.scenePos())
            self.updateTextItemPos()
            if self.xSpinBox is not None and self.ySpinBox is not None:
                self.xSpinBox.setValue(self.pos().x())
                self.ySpinBox.setValue(self.pos().y())

        return super().itemChange(change, value)

    def mousePressEvent(self, mouse_event):
        for item in self.scene().selectedItems():
            item.isDragged = True
        self.isDragged = True

        self.oldMousePos = mouse_event.scenePos()
        self.oldItemPos = self.scenePos()
        # save positions for all items in scene
        for item in self.scene().items():
            if hasattr(item, "oldItemPos"):
                item.oldItemPos = item.scenePos()
        self.oldItemRotation = self.rotation()
        modifier = QtWidgets.QApplication.keyboardModifiers()
        if modifier == QtCore.Qt.KeyboardModifier.ControlModifier:
            self.ctrlPressed = True
        else:
            self.ctrlPressed = False

        return super().mousePressEvent(mouse_event)

    def mouseMoveEvent(self, mouse_event):
        # rotate
        if self.isDragged and self.ctrlPressed:
            diff = mouse_event.scenePos() - self.oldMousePos
            angle = np.arctan2(diff.y(), diff.x())
            angle = 360 * angle / (2 * np.pi)  # convert radians to degrees
            self.setRotation(angle)
            self.updateTextItemPos()
        # translate
        elif self.isDragged:
            diff = mouse_event.scenePos() - self.oldMousePos
            for item in self.scene().selectedItems():
                if hasattr(item, "oldItemPos"):
                    item.setPos(item.oldItemPos + diff)

    def mouseReleaseEvent(self, mouse_event):
        for item in self.scene().selectedItems():
            if hasattr(item, "isDragged"):
                item.isDragged = False
        return super().mouseReleaseEvent(mouse_event)



class WaypointGraphicsEllipseItem(ArenaGraphicsEllipseItem):
    '''
    This item is meant to visualize a waypoint and is connected to a parent WaypointWidget.
    '''
    def __init__(self, waypointWidget, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.waypointWidget = waypointWidget

        # set color
        brush = QtGui.QBrush(QtGui.QColor("blue"), QtCore.Qt.BrushStyle.SolidPattern)
        self.setBrush(brush)

        self.keyPressEater = KeyPressEater(self.handleEvent)

    def setPosNoEvent(self, x, y):
        super().setPosNoEvent(x, y)
        self.waypointWidget.pedsimAgentWidget.drawWaypointPath()

    def itemChange(self, change, value):
        if change == QtWidgets.QGraphicsItem.GraphicsItemChange.ItemPositionChange:
            self.waypointWidget.updateSpinBoxesFromGraphicsItem()
            self.waypointWidget.pedsimAgentWidget.drawWaypointPath()

        return super().itemChange(change, value)

    def handleEvent(self, event):
        if (event.type() == QtCore.QEvent.Type.KeyRelease
            and event.key() == QtCore.Qt.Key.Key_Delete
            and self.isSelected()):
            self.remove()
            return True

        return False

    def remove(self):
        self.waypointWidget.remove()


class SubgoalEllipseItem(ArenaGraphicsEllipseItem):
    '''
    This item is meant to visualize a subgoal and is connected to a parent PathCreator.
    '''
    def __init__(self, pathCreator, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.pathCreator = pathCreator

        # set color
        brush = QtGui.QBrush(QtGui.QColor("blue"), QtCore.Qt.BrushStyle.SolidPattern)
        self.setBrush(brush)

        self.keyPressEater = KeyPressEater(self.handleEvent)

    def setPosNoEvent(self, x, y):
        super().setPosNoEvent(x, y)
        self.pathCreator.drawWaypointPath()

    def itemChange(self, change, value):
        if change == QtWidgets.QGraphicsItem.GraphicsItemChange.ItemPositionChange:
            self.pathCreator.drawWaypointPath()

        return super().itemChange(change, value)

    def handleEvent(self, event):
        if (event.type() == QtCore.QEvent.Type.KeyRelease
            and event.key() == QtCore.Qt.Key.Key_Delete
            and self.isSelected()):
            self.remove()
            return True

        return False

    def remove(self):
        self.pathCreator.removeWaypoint(self)


class ArenaQGraphicsPolygonItem(QtWidgets.QGraphicsPolygonItem):
    '''
    A QGraphicsPolygonItem that is connected to a QGraphicsScene and two QDoubleSpinBoxes
    that hold the position of this item.
    Each vertice of the polygon can be moved by dragging.
    '''
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.setFlags(
            QtWidgets.QGraphicsItem.GraphicsItemFlag.ItemIsSelectable |
            QtWidgets.QGraphicsItem.GraphicsItemFlag.ItemSendsGeometryChanges
            )
        self.setAcceptHoverEvents(True)
        self.footprint_widget = None
        # handles for resizing
        self.handle_size = 0.3  # length of one side of a rectangular handle
        self.handles = []  # list of QRectangle
        self.updateHandlesPos()
        self.point_index = -1

    def handleAt(self, point):
        """
        Returns the index of the resize handle below the given point.
        """
        valid_handles = []
        for i, handle in enumerate(self.handles):
            if handle.contains(point):
                valid_handles.append(i)

        # select handle which center is closest to *point*
        min_diff = float("inf")
        selected_handle_idx = -1
        for handle_idx in valid_handles:
            diff = point - self.handles[handle_idx].center()
            diff_len = np.linalg.norm([diff.x(), diff.y()])
            if diff_len < min_diff:
                min_diff = diff_len
                selected_handle_idx = handle_idx
                
        return selected_handle_idx

    def mousePressEvent(self, mouse_event):
        """
        Executed when the mouse is pressed on the item.
        """
        self.footprint_widget.dragging_polygon = True
        self.point_index = self.handleAt(mouse_event.pos())
        self.mouse_press_pos = mouse_event.pos()
        self.mouse_press_polygon = self.polygon()
        return super().mousePressEvent(mouse_event)

    def mouseMoveEvent(self, mouse_event):
        """
        Executed when the mouse is being moved over the item while being pressed.
        """
        if self.point_index != -1:
            # moving one vertice
            self.interactiveResize(self.point_index, mouse_event.pos())
        else:
            # moving the whole polygon
            for i in range(len(self.polygon())):
                self.interactiveResize(i, mouse_event.pos())

    def mouseReleaseEvent(self, mouse_event):
        """
        Executed when the mouse is released from the item.
        """
        self.footprint_widget.dragging_polygon = False
        self.point_index = -1
        self.mouse_press_pos = None
        self.mouse_press_polygon = None
        self.setPolygon(self.getRoundedPolygon())
        self.footprint_widget.update_spin_boxes()
        return super().mouseReleaseEvent(mouse_event)

    def interactiveResize(self, point_index_, mouse_pos):
        polygon = self.polygon()
        diff = mouse_pos - self.mouse_press_pos
        polygon[point_index_] = self.mouse_press_polygon[point_index_] + diff
        self.setPolygon(polygon)
        self.setPolygon(self.getRoundedPolygon())
        self.footprint_widget.update_spin_boxes()

    def setPolygon(self, *args):
        super().setPolygon(*args)
        self.updateHandlesPos()

    def updateHandlesPos(self):
        d = self.handle_size
        self.handles = []
        for point in self.polygon():
            rect = QtCore.QRectF(point.x() - d / 2.0, point.y() - d / 2.0, d, d)
            self.handles.append(rect)

    def itemChange(self, change, value):
        if change == QtWidgets.QGraphicsItem.GraphicsItemChange.ItemPositionChange:
            self.footprint_widget.update_spin_boxes()

        return super().itemChange(change, value)

    def getRoundedPolygon(self) -> QtGui.QPolygonF:
        mapped_polygon = self.mapToScene(self.polygon())
        for i in range(len(mapped_polygon)):
            mapped_polygon[i] = QtCore.QPointF(round_to_closest_20th(mapped_polygon[i].x()), round_to_closest_20th(mapped_polygon[i].y()))
        rounded_polygon = self.mapFromScene(mapped_polygon)
        return rounded_polygon

    def hoverMoveEvent(self, move_event):
        """
        Executed when the mouse moves over the shape (NOT PRESSED).
        """
        handle = self.handleAt(move_event.pos())
        if handle != -1:
            self.setCursor(QtCore.Qt.SizeFDiagCursor)
        else:
            self.setCursor(QtCore.Qt.CursorShape.SizeAllCursor)

        return super().hoverMoveEvent(move_event)



class ActiveModeWindow(QtWidgets.QMessageBox):
    '''
    A Window that pops up to indicate that a special mode has been activated.
    In this case it's the "Add Waypoints Mode".
    '''
    def __init__(self, connectedWidget, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.connectedWidget = connectedWidget
        self.setIcon(QtWidgets.QMessageBox.Icon.Information)
        self.setWindowTitle("Add Waypoints...")
        self.setText("Click anywhere on the map to add a waypoint.\nPress ESC to finish.")
        self.setWindowModality(QtCore.Qt.WindowModality.NonModal)
        self.setWindowFlag(QtCore.Qt.WindowType.WindowStaysOnTopHint, True)
        self.move(1000, 180)
        self.buttonClicked.connect(self.disable)

    def keyPressEvent(self, event: QtGui.QKeyEvent):
        if event.key() == QtCore.Qt.Key.Key_Escape:
            self.disable()
        return super().keyPressEvent(event)

    def closeEvent(self, event: QtGui.QCloseEvent):
        self.disable()
        return super().closeEvent(event)

    def disable(self):
        self.connectedWidget.addWaypointModeActive = False
        self.hide()



class Line(QtWidgets.QFrame):
    '''
    A simple line that can be added to a layout to separate widgets.
    '''
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.setMinimumWidth(300)
        self.setFrameShape(QtWidgets.QFrame.HLine)
        self.setFrameShadow(QtWidgets.QFrame.Sunken)



class ArenaProbabilitySliderWidget(QtWidgets.QWidget):
    '''
    A Widget containing a slider and a percentage label.
    It has a logarithmic scale to give finer control for smaller probabilities.
    '''
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.value = 0.0
        self.values = np.array([0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
        self.setup_ui()
        self.updateValueFromSlider()

    def setup_ui(self):
        self.setLayout(QtWidgets.QHBoxLayout())
        self.setMinimumWidth(300)

        # slider
        self.slider = QtWidgets.QSlider()
        self.slider.setMinimum(0)
        self.slider.setMaximum(19)
        self.slider.setValue(9)
        self.slider.setTickPosition(QtWidgets.QSlider.TickPosition.TicksAbove)
        self.slider.valueChanged.connect(self.updateValueFromSlider)
        self.slider.setOrientation(QtCore.Qt.Orientation.Horizontal)
        self.layout().addWidget(self.slider)

        # label
        self.label = QtWidgets.QLabel("")
        self.label.setAlignment(QtCore.Qt.AlignmentFlag.AlignRight)
        self.label.setMinimumWidth(50)
        self.layout().addWidget(self.label)

        # unit
        self.unitLabel = QtWidgets.QLabel("%")
        self.unitLabel.setAlignment(QtCore.Qt.AlignmentFlag.AlignRight)
        self.unitLabel.setMinimumWidth(30)
        self.layout().addWidget(self.unitLabel)

    def updateValueFromSlider(self):
        new_value = self.values[self.slider.value()]
        self.value = new_value
        self.label.setText("{:4.2f}".format(new_value * 100))

    def setValue(self, value: float):
        # get absolute differences
        values = np.abs(self.values - value)
        min_idx = np.argmin(values)
        # set slider position
        # actual value and label will be updated by sliders valueChanged signal
        self.slider.setValue(min_idx)

    def getValue(self):
        return self.value



class ArenaSliderWidget(QtWidgets.QWidget):
    '''
    A Widget containing a slider and labels to display the value and a unit.
    '''
    def __init__(self, minValue: int, numValues: int, stepValue: float, unit: str = "", **kwargs):
        super().__init__(**kwargs)
        assert isinstance(minValue, int)
        assert isinstance(numValues, int)
        self.minValue = minValue
        self.numValues = numValues
        self.stepValue = stepValue
        self.unit = unit
        self.value = 0.0
        self.setup_ui()
        self.udpateValueFromSlider()

    def setup_ui(self):
        self.setLayout(QtWidgets.QHBoxLayout())
        self.setMinimumWidth(300)

        # slider
        self.slider = QtWidgets.QSlider()
        self.slider.setMinimum(self.minValue)
        self.slider.setMaximum(self.minValue + self.numValues)
        self.slider.setValue(self.minValue + (self.numValues // 2))
        self.slider.setTickPosition(QtWidgets.QSlider.TickPosition.TicksAbove)
        self.slider.valueChanged.connect(self.udpateValueFromSlider)
        self.slider.setOrientation(QtCore.Qt.Orientation.Horizontal)
        self.layout().addWidget(self.slider)

        # label
        self.label = QtWidgets.QLabel("")
        self.label.setAlignment(QtCore.Qt.AlignmentFlag.AlignRight)
        self.label.setMinimumWidth(50)
        self.layout().addWidget(self.label)

        # unit
        self.unitLabel = QtWidgets.QLabel(self.unit)
        self.unitLabel.setAlignment(QtCore.Qt.AlignmentFlag.AlignRight)
        self.unitLabel.setMinimumWidth(30)
        self.layout().addWidget(self.unitLabel)

    def udpateValueFromSlider(self):
        new_value = self.slider.value() * self.stepValue
        self.value = new_value
        self.label.setText("{:4.2f}".format(new_value))

    def setValue(self, value: float):
        # construct list of actual possible values for this slider
        values = (np.arange(self.numValues + 1) * self.stepValue) + self.minValue
        # get absolute differences
        abs_diffs = np.abs(values - value)
        min_idx = np.argmin(abs_diffs)
        # set slider position
        # actual value and label will be updated by sliders valueChanged signal
        self.slider.setValue(min_idx)

    def getValue(self):
        return self.value



class ArenaQDoubleSpinBox(QtWidgets.QDoubleSpinBox):
    '''
    A QDoubleSpinBox where a single step is 0.1 and the value will be rounded on every step up or down.
    '''
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.setSingleStep(0.1)
        self.setMinimum(-100.0)
        self.setValue(0.0)

    def wheelEvent(self, event):
        angle = event.angleDelta().y()
        if angle > 0:
            self.stepUp()
            event.accept()
        elif angle < 0:
            self.stepDown()
            event.accept()
        else:
            event.ignore()

    def stepUp(self):
        new_value = round(self.value() + self.singleStep(), 1)
        self.setValue(new_value)

    def stepDown(self):
        new_value = round(self.value() - self.singleStep(), 1)
        self.setValue(new_value)



class ArenaQGraphicsScene(QtWidgets.QGraphicsScene):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def removeSelected(self):
        for item in self.selectedItems():
            if isinstance(item, ArenaGraphicsPathItem):
                item.remove()



class ArenaQGraphicsView(QtWidgets.QGraphicsView):
    '''
    A custom QGraphicsView.
    - can be dragged by mouse
    - can be zoomed by mouse wheel
    - sends mouse click positions (except clicks from dragging)
    '''
    clickedPos = QtCore.pyqtSignal(QtCore.QPointF)

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.setDragMode(QtWidgets.QGraphicsView.DragMode.ScrollHandDrag)
        self.setHorizontalScrollBarPolicy(QtCore.Qt.ScrollBarPolicy.ScrollBarAlwaysOn)
        self.setVerticalScrollBarPolicy(QtCore.Qt.ScrollBarPolicy.ScrollBarAlwaysOn)
        self.setSceneRect(-200, -200, 400, 400)
        self.zoomFactor = 1.0
        self.setViewportUpdateMode(QtWidgets.QGraphicsView.ViewportUpdateMode.FullViewportUpdate)

        # add coordinate system lines
        pen = QtGui.QPen()
        pen.setWidthF(0.01)
        self.scene().addLine(0, -100, 0, 100, pen)
        self.scene().addLine(-100, 0, 100, 0, pen)

        # set initial view
        rect = QtCore.QRectF(-1.5, -1.5, 3, 3)
        self.fitInView(rect, mode=QtCore.Qt.AspectRatioMode.KeepAspectRatio)

        self.lastMousePos = QtCore.QPointF()

    def mousePressEvent(self, event: QtGui.QMouseEvent):
        self.lastMousePos = event.pos()
        if event.buttons() & QtCore.Qt.MouseButton.RightButton:
            self.setDragMode(QtWidgets.QGraphicsView.DragMode.RubberBandDrag)
        if event.buttons() & QtCore.Qt.MouseButton.LeftButton:
            self.setDragMode(QtWidgets.QGraphicsView.DragMode.ScrollHandDrag)

        return super().mousePressEvent(event)

    def mouseReleaseEvent(self, event):
        # only emit signal if mouse was not dragged
        diff = event.pos() - self.lastMousePos
        diff_len = diff.x() + diff.y()
        if abs(diff_len) < 2.0:
            pos = self.mapToScene(event.pos())
            self.clickedPos.emit(pos)
        return super().mouseReleaseEvent(event)

    def wheelEvent(self, event):
        """
        Zoom in or out of the view.
        """
        zoomInFactor = 1.25
        zoomOutFactor = 1 / zoomInFactor

        # Save the scene pos
        oldPos = self.mapToScene(event.pos())

        # Zoom
        if event.angleDelta().y() > 0:
            zoomFactor_ = zoomInFactor
        else:
            zoomFactor_ = zoomOutFactor
        self.scale(zoomFactor_, zoomFactor_)
        self.zoomFactor *= 1 / zoomFactor_

        # Get the new position
        newPos = self.mapToScene(event.pos())

        # Move scene to old position
        delta = newPos - oldPos
        self.translate(delta.x(), delta.y())

In [2]:
def get_ros_package_path(package_name: str) -> str:
    try:
        import rospkg

        rospack = rospkg.RosPack()
        return rospack.get_path(package_name)
    except:
        return ""


def get_nth_decimal_part(x: float, n: int) -> int:
    """
    Get the n'th decimal part of a decimal number.
    Example:
        get_nth_decimal_part(1.234, 2) == 3
    """
    x *= 10 ** n  # push relevant part directly in front of decimal point
    x %= 10  # remove parts left of the relevant part
    return int(x)  # remove decimal places


def round_to_closest_20th(x: float) -> float:
    """
    Round to X.X0 or X.X5.
    Example:
        round_one_and_half_decimal_places(1.234) == 1.25
    """
    return round(x * 20) / 20

def rad_to_deg(angle: float) -> float:
    import math
    angle = normalize_angle_rad(angle)
    angle = 360.0 * angle / (2.0 * math.pi)
    return angle

def deg_to_rad(angle: float) -> float:
    import math
    angle = normalize_angle_deg(angle)
    angle = 2 * math.pi * angle / 360.0
    return angle

def normalize_angle_deg(angle: float) -> float:
    import math

    # make sure angle is positive
    while angle < 0:
        angle += 360

    # make sure angle is between 0 and 360
    angle = math.fmod(angle, 360.0)
    return angle

def normalize_angle_rad(angle: float) -> float:
    import math

    # make sure angle is positive
    while angle < 0:
        angle += 2 * math.pi
    # make sure angle is between 0 and 2 * pi
    angle = math.fmod(angle, 2 * math.pi)
    return angle

def normalize_angle(angle: float, rad: bool = True) -> float:
    if rad:
        return normalize_angle_rad(angle)
    else:
        return normalize_angle_deg(angle)

def get_current_user_path(path_in: str) -> str:
    """
    Convert a path from another user to the current user, for example:
    "/home/alice/catkin_ws" -> "/home/bob/catkin_ws"
    """
    if path_in == "":
        return ""
    from pathlib import Path

    path = Path(path_in)
    new_path = Path.home().joinpath(*path.parts[3:])
    return str(new_path)

def remove_file_ending(file_name: str) -> str:
    """
    Remove everything after the first "." in a string.
    """
    file_ending_index = file_name.find(".")
    if file_ending_index != -1:
        return file_name[:file_ending_index]
    return file_name

In [3]:
from PyQt5 import QtGui, QtCore, QtWidgets
import numpy as np
import os
import yaml
import shutil
import pathlib
import re
import subprocess
from typing import List
from PIL import Image
from enum import Enum
# from HelperFunctions import *
# from QtExtensions import *


class MapType(Enum):
    INDOOR = 0
    OUTDOOR = 1


class MapGenerator(QtWidgets.QMainWindow):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        # add graphicsscene and graphicsview
        self.scene = QtWidgets.QGraphicsScene()
        self.view = QtWidgets.QGraphicsView(self.scene)
        self.view.setDragMode(QtWidgets.QGraphicsView.DragMode.ScrollHandDrag)
        self.view.setSceneRect(-1000, -1000, 2000, 2000)
        self.view.fitInView(QtCore.QRectF(-1.5, -1.5, 100, 100),
                            mode=QtCore.Qt.AspectRatioMode.KeepAspectRatio)
        self.view.setHorizontalScrollBarPolicy(
            QtCore.Qt.ScrollBarPolicy.ScrollBarAlwaysOff)

        self.setup_ui()
        self.updateWidgetsFromSelectedType(self.type_dropdown.currentIndex())
        self.showPreview()

    def setup_ui(self):
        self.setWindowTitle("Map Generator")
        self.resize(1100, 600)
        self.move(100, 100)
        icon = QtGui.QIcon()
        icon.addPixmap(QtGui.QPixmap('icon.png'),
                       QtGui.QIcon.Selected, QtGui.QIcon.On)
        self.setWindowIcon(icon)
        layout_index = 0

        # set central widget
        central_widget = QtWidgets.QWidget()
        central_widget.setLayout(QtWidgets.QGridLayout())
        self.setCentralWidget(central_widget)

        # splitter
        self.splitter = QtWidgets.QSplitter()
        self.centralWidget().layout().addWidget(self.splitter)

        # left side frame
        frame = QtWidgets.QFrame()
        frame.setFrameStyle(QtWidgets.QFrame.Shape.Box |
                            QtWidgets.QFrame.Shadow.Raised)
        frame.setLayout(QtWidgets.QGridLayout())
        frame.setSizePolicy(QtWidgets.QSizePolicy.Policy.Maximum,
                            QtWidgets.QSizePolicy.Policy.Maximum)
        self.splitter.addWidget(frame)

        # width
        # label
        width_label = QtWidgets.QLabel("### Width")
        width_label.setTextFormat(QtCore.Qt.TextFormat.MarkdownText)
        frame.layout().addWidget(width_label, layout_index,
                                 0, QtCore.Qt.AlignmentFlag.AlignLeft)
        # spinbox
        self.width_spin_box = QtWidgets.QSpinBox()
        self.width_spin_box.setRange(1, 10000)
        self.width_spin_box.setValue(101)
        self.width_spin_box.setSingleStep(1)
        self.width_spin_box.setFixedSize(150, 30)
        self.width_spin_box.valueChanged.connect(self.showPreview)
        frame.layout().addWidget(self.width_spin_box, layout_index,
                                 1, QtCore.Qt.AlignmentFlag.AlignRight)
        layout_index += 1

        # height
        # label
        height_label = QtWidgets.QLabel("### Height")
        height_label.setTextFormat(QtCore.Qt.TextFormat.MarkdownText)
        frame.layout().addWidget(height_label, layout_index,
                                 0, QtCore.Qt.AlignmentFlag.AlignLeft)
        # spinbox
        self.height_spin_box = QtWidgets.QSpinBox()
        self.height_spin_box.setRange(1, 10000)
        self.height_spin_box.setValue(101)
        self.height_spin_box.setSingleStep(1)
        self.height_spin_box.setFixedSize(150, 30)
        self.height_spin_box.valueChanged.connect(self.showPreview)
        frame.layout().addWidget(self.height_spin_box, layout_index,
                                 1, QtCore.Qt.AlignmentFlag.AlignRight)
        layout_index += 1

        # type
        # label
        type_label = QtWidgets.QLabel("### Type")
        type_label.setTextFormat(QtCore.Qt.TextFormat.MarkdownText)
        frame.layout().addWidget(type_label, layout_index,
                                 0, QtCore.Qt.AlignmentFlag.AlignLeft)
        # dropdown
        self.type_dropdown = QtWidgets.QComboBox()
        for map_type in MapType:
            self.type_dropdown.insertItem(
                map_type.value, map_type.name.lower())
        self.type_dropdown.setFixedSize(150, 30)
        self.type_dropdown.currentIndexChanged.connect(self.handleTypeChanged)
        frame.layout().addWidget(self.type_dropdown, layout_index,
                                 1, QtCore.Qt.AlignmentFlag.AlignRight)
        layout_index += 1

        # corridor width
        # label
        self.corridor_width_label = QtWidgets.QLabel("### Corridor Width")
        self.corridor_width_label.setTextFormat(
            QtCore.Qt.TextFormat.MarkdownText)
        frame.layout().addWidget(self.corridor_width_label,
                                 layout_index, 0, QtCore.Qt.AlignmentFlag.AlignLeft)
        # spinbox
        self.corridor_width_spin_box = QtWidgets.QSpinBox()
        self.corridor_width_spin_box.setRange(0, 1000)
        self.corridor_width_spin_box.setValue(3)
        self.corridor_width_spin_box.setSingleStep(1)
        self.corridor_width_spin_box.setFixedSize(150, 30)
        self.corridor_width_spin_box.valueChanged.connect(self.showPreview)
        frame.layout().addWidget(self.corridor_width_spin_box,
                                 layout_index, 1, QtCore.Qt.AlignmentFlag.AlignRight)
        layout_index += 1

        # iterations
        # label
        self.iterations_label = QtWidgets.QLabel("### Iterations")
        self.iterations_label.setTextFormat(QtCore.Qt.TextFormat.MarkdownText)
        frame.layout().addWidget(self.iterations_label, layout_index,
                                 0, QtCore.Qt.AlignmentFlag.AlignLeft)
        # spinbox
        self.iterations_spin_box = QtWidgets.QSpinBox()
        self.iterations_spin_box.setRange(0, 1000)
        self.iterations_spin_box.setValue(100)
        self.iterations_spin_box.setSingleStep(1)
        self.iterations_spin_box.setFixedSize(150, 30)
        self.iterations_spin_box.valueChanged.connect(self.showPreview)
        frame.layout().addWidget(self.iterations_spin_box, layout_index,
                                 1, QtCore.Qt.AlignmentFlag.AlignRight)
        layout_index += 1

        # obstacles
        # label
        self.obstacles_label = QtWidgets.QLabel("### Obstacles")
        self.obstacles_label.setTextFormat(QtCore.Qt.TextFormat.MarkdownText)
        frame.layout().addWidget(self.obstacles_label, layout_index,
                                 0, QtCore.Qt.AlignmentFlag.AlignLeft)
        # spinbox
        self.obstacles_spin_box = QtWidgets.QSpinBox()
        self.obstacles_spin_box.setRange(0, 1000)
        self.obstacles_spin_box.setValue(20)
        self.obstacles_spin_box.setSingleStep(1)
        self.obstacles_spin_box.setFixedSize(150, 30)
        self.obstacles_spin_box.valueChanged.connect(self.showPreview)
        frame.layout().addWidget(self.obstacles_spin_box, layout_index,
                                 1, QtCore.Qt.AlignmentFlag.AlignRight)
        layout_index += 1

        # obstacle size
        # label
        self.obstacle_size_label = QtWidgets.QLabel("### Obstacle Size")
        self.obstacle_size_label.setTextFormat(
            QtCore.Qt.TextFormat.MarkdownText)
        frame.layout().addWidget(self.obstacle_size_label,
                                 layout_index, 0, QtCore.Qt.AlignmentFlag.AlignLeft)
        # spinbox
        self.obstacle_size_spin_box = QtWidgets.QSpinBox()
        self.obstacle_size_spin_box.setRange(0, 1000)
        self.obstacle_size_spin_box.setValue(2)
        self.obstacle_size_spin_box.setSingleStep(1)
        self.obstacle_size_spin_box.setFixedSize(150, 30)
        self.obstacle_size_spin_box.valueChanged.connect(self.showPreview)
        frame.layout().addWidget(self.obstacle_size_spin_box,
                                 layout_index, 1, QtCore.Qt.AlignmentFlag.AlignRight)
        layout_index += 1

        # line
        line = Line()
        frame.layout().addWidget(line, layout_index, 0, 1, -1)
        layout_index += 1

        # number of maps
        # label
        number_of_maps_label = QtWidgets.QLabel("### Number of Maps")
        number_of_maps_label.setTextFormat(QtCore.Qt.TextFormat.MarkdownText)
        frame.layout().addWidget(number_of_maps_label, layout_index,
                                 0, QtCore.Qt.AlignmentFlag.AlignLeft)
        # spinbox
        self.number_of_maps_spin_box = QtWidgets.QSpinBox()
        self.number_of_maps_spin_box.setRange(0, 1000)
        self.number_of_maps_spin_box.setValue(5)
        self.number_of_maps_spin_box.setSingleStep(1)
        self.number_of_maps_spin_box.setFixedSize(150, 30)
        self.number_of_maps_spin_box.valueChanged.connect(
            self.numberOfMapsChanged)
        frame.layout().addWidget(self.number_of_maps_spin_box,
                                 layout_index, 1, QtCore.Qt.AlignmentFlag.AlignRight)
        layout_index += 1

        # resolution
        # label
        resolution_label = QtWidgets.QLabel("### Map Resolution")
        resolution_label.setTextFormat(QtCore.Qt.TextFormat.MarkdownText)
        frame.layout().addWidget(resolution_label, layout_index,
                                 0, QtCore.Qt.AlignmentFlag.AlignLeft)
        # spinbox
        self.resolution_spin_box = QtWidgets.QDoubleSpinBox()
        self.resolution_spin_box.setRange(0, 1000)
        self.resolution_spin_box.setValue(0.5)
        self.resolution_spin_box.setSingleStep(0.01)
        self.resolution_spin_box.setFixedSize(150, 30)
        frame.layout().addWidget(self.resolution_spin_box, layout_index,
                                 1, QtCore.Qt.AlignmentFlag.AlignRight)
        layout_index += 1

        # folder
        folder_label = QtWidgets.QLabel("Save to Folder:")
        frame.layout().addWidget(folder_label, layout_index,
                                 0, QtCore.Qt.AlignmentFlag.AlignLeft)
        browse_button = QtWidgets.QPushButton("Browse...")
        browse_button.clicked.connect(self.onBrowseClicked)
        frame.layout().addWidget(browse_button, layout_index,
                                 1, QtCore.Qt.AlignmentFlag.AlignRight)
        layout_index += 1
        self.folder_edit = QtWidgets.QLineEdit("Please select folder")
        # set default path to simulator_setup/maps if it exists
        sim_setup_path = get_ros_package_path("simulator_setup")
        if sim_setup_path != "":
            self.folder_edit.setText(os.path.join(sim_setup_path, "maps"))
        frame.layout().addWidget(self.folder_edit, layout_index, 0, 1, -1)
        layout_index += 1

        # generate maps button
        self.generate_maps_button = QtWidgets.QPushButton("Generate 5 maps")
        self.generate_maps_button.clicked.connect(self.onGenerateMapsClicked)
        frame.layout().addWidget(self.generate_maps_button, layout_index, 0, 1, -1)
        layout_index += 1

        # generate maps result label
        self.result_label = QtWidgets.QLabel("")
        self.result_label.setMaximumHeight(50)
        frame.layout().addWidget(self.result_label, layout_index, 0, 1, -1)
        layout_index += 1

        # generate maps result label animation
        self.result_label_animation = QtCore.QPropertyAnimation(
            self, b"text_color", self)
        self.result_label_animation.setDuration(250)
        self.result_label_animation.setLoopCount(2)
        self.result_label_animation.setStartValue(QtGui.QColor(255, 255, 255))
        self.result_label_animation.setEndValue(QtGui.QColor(0, 0, 0))

        # right side graphicsview
        self.splitter.addWidget(self.view)

        # set splitter sizes
        self.splitter.setSizes([300, 700])

    def handleTypeChanged(self):
        self.showPreview()
        self.updateWidgetsFromSelectedType(self.type_dropdown.currentIndex())

    def getTextColor(self) -> QtGui.QColor:
        return self.result_label.palette().text().color()

    def setTextColor(self, color: QtGui.QColor):
        palette = self.result_label.palette()
        palette.setColor(self.result_label.foregroundRole(), color)
        self.result_label.setPalette(palette)

    # text color property
    text_color = QtCore.pyqtProperty(QtGui.QColor, getTextColor, setTextColor)

    def numberOfMapsChanged(self, value):
        s = f"Generate {value} maps"
        self.generate_maps_button.setText(s)

    def getMapNames(self) -> List[str]:
        '''
        Generate simple map names that don't exist yet in the form of f"map{index}".
        Search the maps folder for already existing maps in this format. Get the highest index and then
        start counting from there.
        '''
        folder = pathlib.Path(self.folder_edit.text())
        map_folders = [p for p in folder.iterdir() if p.is_dir()]
        names = [p.parts[-1] for p in map_folders]
        # get only the names that are in the form of f"map{index}"
        prefix = "map"
        pat = re.compile(f"{prefix}\d+$", flags=re.ASCII)
        filtered_names = [name for name in names if pat.match(name) != None]
        # get the max index that already exists
        max_index = 0
        if len(filtered_names) > 0:
            max_index = max([int(name[len(prefix):])
                            for name in filtered_names])
        number_of_maps = self.number_of_maps_spin_box.value()
        # generate new names beginning with the max index
        return [f"map{i}" for i in range(max_index+1, max_index+1+number_of_maps)]

    def onGenerateMapsClicked(self):
        # generate maps
        height = self.height_spin_box.value()
        width = self.width_spin_box.value()
        path = pathlib.Path(self.folder_edit.text())

        # create new maps with appropriate names
        map_names = self.getMapNames()
        for map_name in map_names:
            map_array,x,y = self.getCurrentMap()
            if map_array is not None:
                self.make_image(map_array,x,y, path, map_name)
                self.create_yaml_files(path / map_name)

        # update result text
        if len(map_names) > 0:
            if len(map_names) == 1:
                self.result_label.setText(
                    f"Generated {len(map_names)} map: {map_names[0]}")
            elif len(map_names) > 1:
                self.result_label.setText(
                    f"Generated {len(map_names)} maps: {map_names[0]} - {map_names[-1]}")

            self.result_label_animation.start()

        # display maps in scene
        # remove old maps
        items = self.scene.items()
        for item in items:
            self.scene.removeItem(item)
        # add new maps
        offset_hor = 0
        offset_ver = 0
        for map_name in map_names:
            image_path = path / map_name / f"{map_name}.png"
            pixmap = QtGui.QPixmap(str(image_path))
            pixmap_item = QtWidgets.QGraphicsPixmapItem(pixmap)
            pixmap_item.setOffset(offset_hor, offset_ver)
            self.scene.addItem(pixmap_item)
            offset_ver += height + 10
        self.view.fitInView(QtCore.QRectF(-5, -5, width + 5, height +
                            (height / 3)), mode=QtCore.Qt.AspectRatioMode.KeepAspectRatio)

    def getCurrentMap(self) :
        map_type = MapType(self.type_dropdown.currentIndex())
        height = self.height_spin_box.value()
        width = self.width_spin_box.value()
        if height < 10 or width < 10:
            return None
        map_array = None
        x = y = None
        if map_type == MapType.INDOOR:
            corridor_radius = self.corridor_width_spin_box.value()
            iterations = self.iterations_spin_box.value()
            map_array,x,y = self.create_indoor_map(
                height, width, corridor_radius, iterations)
            
        elif map_type == MapType.OUTDOOR:
            obstacle_number = self.obstacles_spin_box.value()
            obstacle_extra_radius = self.obstacle_size_spin_box.value()
            map_array,x,y = self.create_outdoor_map(
                height, width, obstacle_number, obstacle_extra_radius)

        return (map_array,x,y)

    def getXpmFromNdarray(self, a: np.ndarray) -> List[str]:
        height, width = a.shape
        xpm = []
        xpm.append(f"{width} {height} 2 1")
        xpm.append("1 c #000000")
        xpm.append("0 c #FFFFFF")
        for i in range(height):
            line = ""
            for j in range(width):
                line += str(int(a[i, j]))
            xpm.append(line)
        return xpm

    def showPreview(self):
        '''
        Generate an example map with the current settings and display it.
        '''
        # clear scene
        items = self.scene.items()
        for item in items:
            self.scene.removeItem(item)

        # generate a map
        map_array,x,y = self.getCurrentMap()

        if map_array is None:
            return

        # add map to the scene
        # generate XPM data from array
        xpm = self.getXpmFromNdarray(map_array)
        # get pixmap from XPM data
        pixmap = QtGui.QPixmap(xpm)
        pixmap_item = QtWidgets.QGraphicsPixmapItem(pixmap)
        # add to scene
        self.scene.addItem(pixmap_item)
        # adjust view
        height = self.height_spin_box.value()
        width = self.width_spin_box.value()
        self.view.fitInView(QtCore.QRectF(-5, -5, width + 5, height +
                            (height / 3)), mode=QtCore.Qt.AspectRatioMode.KeepAspectRatio)

    def onBrowseClicked(self):
        path = QtWidgets.QFileDialog.getExistingDirectory(
            self, "Select Maps Folder", str(pathlib.Path.home()))
        if os.path.exists(path):
            self.folder_edit.setText(path)

    def updateWidgetsFromSelectedType(self, index):
        map_type = MapType(index)
        if map_type == MapType.INDOOR:
            self.corridor_width_label.show()
            self.corridor_width_spin_box.show()
            self.iterations_label.show()
            self.iterations_spin_box.show()
            self.obstacles_label.hide()
            self.obstacles_spin_box.hide()
            self.obstacle_size_label.hide()
            self.obstacle_size_spin_box.hide()
        elif map_type == MapType.OUTDOOR:
            self.corridor_width_label.hide()
            self.corridor_width_spin_box.hide()
            self.iterations_label.hide()
            self.iterations_spin_box.hide()
            self.obstacles_label.show()
            self.obstacles_spin_box.show()
            self.obstacle_size_label.show()
            self.obstacle_size_spin_box.show()

    def create_yaml_files(self, map_folder_path: pathlib.Path):
        '''
        Create the files map.yaml (ROS) and map.wordl.yaml (Flatland) for the map.
        map_folder_path: path to folder for this map e.g.: /home/user/catkin_ws/src/arena-rosnav/simulator_setup/maps/mymap
        '''
        map_folder = pathlib.Path(map_folder_path)
        map_name = map_folder.parts[-1]

        # create map.yaml
        map_yaml = {
            "image": "{0}.png".format(map_name),
            "resolution": self.resolution_spin_box.value(),
            "origin": [0.0, 0.0, 0.0],  # [-x,-y,0.0]
            "negate": 0,
            "occupied_thresh": 0.65,
            "free_thresh": 0.196
            
        }

        with open(str(map_folder / "map.yaml"), 'w') as outfile:
            yaml.dump(map_yaml, outfile, sort_keys=False,
                      default_flow_style=None)

        # create map.world.yaml
        world_yaml_properties = {
            "properties": {
                "velocity_iterations": 10,
                "position_iterations": 10
            }
        }

        world_yaml_layers = {
            "layers": [
                {
                    "name": "static",
                    "map": "map.yaml",
                    "color": [0, 1, 0, 1]
                }
            ]
        }

        with open(str(map_folder / "map.world.yaml"), 'w') as outfile:
            # somehow the first part must be with default_flow_style=False
            yaml.dump(world_yaml_properties, outfile,
                      sort_keys=False, default_flow_style=False)
            # 2nd part must be with default_flow_style=None
            yaml.dump(world_yaml_layers, outfile,
                      sort_keys=False, default_flow_style=None)

    def make_image(self, map: np.ndarray,x:  int,y:  int ,maps_folder_path: pathlib.Path, map_name: str):
        '''
        Create PNG file from occupancy map (1:occupied, 0:free) and the necessary yaml files.
        - map: numpy array
        - maps_folder_path: path to maps folder e.g.: /home/user/catkin_ws/src/arena-rosnav/simulator_setup/maps
        - map_name: name of map, a folder will be created using this name
        '''
        # create new directory for map
        map_folder = maps_folder_path / map_name
        if not map_folder.exists():
            os.mkdir(str(map_folder))
        # create image
        # monochromatic image
        img = Image.fromarray(((map-1)**2*255).astype('uint8'))
        imgrgb = img.convert('RGB')
        # save image
        # save map in map directory
        imgrgb.save(str(map_folder / (map_name + ".png")))
        f= open(os.path.join(map_folder,"parameters.txt"),"w")
        f.write(str(x)+"\n")
        f.write(str(y)+"\n")
        f.close()

    # create empty map with format given by height,width and initialize empty tree
    def initialize_map(self, height, width, type="indoor"):
        if type == "outdoor":
            map = np.tile(1, [height, width])
            map[slice(1, height-1), slice(1, width-1)] = 0
            return map
        else:
            return np.tile(1, [height, width])

    def insert_root_node(self, map, tree):  # create root node in center of map
        root_node = [int(np.floor(map.shape[0]/2)),
                     int(np.floor(map.shape[1]/2))]
        map[root_node[0], root_node[1]] = 0
        tree.append(root_node)

    # sample position from map within boundary and leave tolerance for corridor width
    def sample(self, map, corridor_radius):
        random_x = np.random.choice(
            range(corridor_radius+2, map.shape[0]-corridor_radius-1, 1))
        random_y = np.random.choice(
            range(corridor_radius+2, map.shape[1]-corridor_radius-1, 1))
        return [random_x, random_y]

    # find nearest node according to L1 norm
    def find_nearest_node(self, random_position, tree):
        nearest_node = []
        min_distance = np.inf
        for node in tree:
            distance = sum(np.abs(np.array(random_position)-np.array(node)))
            if distance < min_distance:
                min_distance = distance
                nearest_node = node
        return nearest_node

    # insert new node into the map and tree
    def insert_new_node(self, random_position, tree, map):
        map[random_position[0], random_position[1]] = 0
        tree.append(random_position)

    def get_constellation(self, node1, node2):
        # there are two relevant constellations for the 2 nodes, which must be considered when creating the horizontal and vertical path
        # 1: lower left and upper right
        # 2: upper left and lower right
        # each of the 2 constellation have 2 permutations which must be considered as well
        constellation1 = {
            # x1>x2 and y1<y2
            "permutation1": node1[0] > node2[0] and node1[1] < node2[1],
            "permutation2": node1[0] < node2[0] and node1[1] > node2[1]}  # x1<x2 and y1>y2
        if constellation1["permutation1"] or constellation1["permutation2"]:
            return 1
        else:
            return 2

    def create_path(self, node1, node2, corridor_radius, map):
        coin_flip = np.random.random()
        # x and y coordinates must be sorted for usage with range function
        x1, x2 = sorted([node1[0], node2[0]])
        y1, y2 = sorted([node1[1], node2[1]])
        if self.get_constellation(node1, node2) == 1:  # check which constellation
            # randomly determine the curvature of the path (right turn/left turn)
            if coin_flip >= 0.5:
                map[slice(x1-corridor_radius, x1+corridor_radius+1), range(y1 -
                                                                           corridor_radius, y2+1+corridor_radius, 1)] = 0  # horizontal path
                map[range(x1-corridor_radius, x2+1+corridor_radius, 1), slice(y1 -
                                                                              corridor_radius, y1+corridor_radius+1)] = 0  # vertical path
            else:
                map[slice(x2-corridor_radius, x2+corridor_radius+1), range(y1 -
                                                                           corridor_radius, y2+1+corridor_radius, 1)] = 0  # horizontal path
                map[range(x1-corridor_radius, x2+1+corridor_radius, 1), slice(y2 -
                                                                              corridor_radius, y2+corridor_radius+1)] = 0  # vertical path
        else:
            # randomly determine the curvature of the path (right turn/left turn)
            if coin_flip >= 0.5:
                map[slice(x1-corridor_radius, x1+corridor_radius+1), range(y1 -
                                                                           corridor_radius, y2+1+corridor_radius, 1)] = 0  # horizontal path
                map[range(x1-corridor_radius, x2+1+corridor_radius, 1), slice(y2 -
                                                                              corridor_radius, y2+corridor_radius+1)] = 0  # vertical path
            else:
                map[slice(x2-corridor_radius, x2+corridor_radius+1), range(y1 -
                                                                           corridor_radius, y2+1+corridor_radius, 1)] = 0  # horizontal path
                map[range(x1-corridor_radius, x2+1+corridor_radius, 1), slice(y1 -
                                                                              corridor_radius, y1+corridor_radius+1)] = 0  # vertical path

    def create_indoor_map(self, height, width, corridor_radius, iterations):
        tree = []  # initialize empty tree
        map = self.initialize_map(height, width)
        self.insert_root_node(map, tree)
        for i in range(iterations):  # create as many paths/nodes as defined in iteration
            random_position = self.sample(map, corridor_radius)
            # nearest node must be found before inserting the new node into the tree, else nearest node will be itself
            nearest_node = self.find_nearest_node(random_position, tree)
            self.insert_new_node(random_position, tree, map)
            self.create_path(random_position, nearest_node,
                             corridor_radius, map)
        return map,iterations,corridor_radius

    def create_outdoor_map(self, height, width, obstacle_number, obstacle_extra_radius):
        map = self.initialize_map(height, width, type="outdoor")
        for i in range(obstacle_number):
            random_position = self.sample(map, obstacle_extra_radius)
            map[slice(random_position[0]-obstacle_extra_radius, random_position[0]+obstacle_extra_radius+1),  # create 1 pixel obstacles with extra radius if specified
                slice(random_position[1]-obstacle_extra_radius, random_position[1]+obstacle_extra_radius+1)] = 1
        return map,obstacle_number,obstacle_extra_radius


if __name__ == "__main__":
    app = QtWidgets.QApplication([])

    widget = MapGenerator()
    widget.show()

    app.exec()